In [ ]:
# Example from Mining the Social Web, Section 6

### Enron email database ###

In [1]:
from functions import *

Enron database statistics:

In [4]:
mongo_cmd('enron', 'db.mbox.stats()')

2016-11-05T22:59:00.299+0200 I CONTROL  [main] Hotfix KB2731284 or later update is installed, no need to zero-out data files
MongoDB shell version: 3.2.8
connecting to: enron
{
	"ns" : "enron.mbox",
	"count" : 41299,
	"size" : 219359952,
	"avgObjSize" : 5311,
	"numExtents" : 12,
	"storageSize" : 222146560,
	"lastExtentSize" : 62554112,
	"paddingFactor" : 1,
	"paddingFactorNote" : "paddingFactor is unused and unmaintained in 3.0. It remains hard coded to 1.0 for compatibility only.",
	"userFlags" : 1,
	"capped" : false,
	"nindexes" : 0,
	"totalIndexSize" : 0,
	"indexSizes" : {
		
	},
	"ok" : 1
}



Message counts by year in the database

In [5]:
client = pymongo.MongoClient()
db = client.enron
mbox = db.mbox
results = list(mbox.aggregate([
    {"$project": {
        "_id": 0,
        "DateBucket": {
            "year": {"$year": "$Date"},
            "month": {"$month": "$Date"},
            "day": {"$dayOfMonth": "$Date"},
            "hour": {"$hour": "$Date"},
        }
    }
    },
    {"$group": {
        "_id": {"year": "$DateBucket.year", "month": "$DateBucket.month"},
        "num_msgs": {"$sum": 1}
    }
    },
    {"$sort": {"_id.year": 1, "_id.month": 1} }
]))

pt = PrettyTable(field_names=['Year', 'Month', 'Num Msgs'])
pt.align['Num Msgs'], pt.align['Month'] = 'r', 'r'
[pt.add_row([result['_id']['year'], result['_id']['month'], result['num_msgs']]) for result in results]
print pt

+------+-------+----------+
| Year | Month | Num Msgs |
+------+-------+----------+
| 1997 |     1 |        1 |
| 1998 |     1 |        1 |
| 2000 |    12 |        1 |
| 2001 |     1 |        3 |
| 2001 |     2 |        3 |
| 2001 |     3 |       21 |
| 2001 |     4 |      811 |
| 2001 |     5 |     2118 |
| 2001 |     6 |     1650 |
| 2001 |     7 |      802 |
| 2001 |     8 |     1538 |
| 2001 |     9 |     3538 |
| 2001 |    10 |    10630 |
| 2001 |    11 |     9219 |
| 2001 |    12 |     4541 |
| 2002 |     1 |     3611 |
| 2002 |     2 |     1919 |
| 2002 |     3 |      514 |
| 2002 |     4 |       97 |
| 2002 |     5 |       85 |
| 2002 |     6 |      166 |
| 2002 |    10 |        1 |
| 2002 |    12 |        1 |
| 2004 |     2 |       26 |
| 2020 |    12 |        2 |
+------+-------+----------+


The number of messages in the collection

In [6]:
mbox = db.mbox
mbox.count()

41299

Display the message as pretty-printed JSON. The use of the custom serializer supplied by PyMongo is necessary in order to handle the date field that is provided as a datetime.datetime tuple.

In [10]:
print(json.dumps(mbox.find_one(), indent=1, default=json_util.default))

{
 "X-cc": "", 
 "From": "customerservice.1556092.33252.0@service.williams-sonoma.com", 
 "Subject": "Fresh Ideas for Easter Brunch from Williams-Sonoma", 
 "To": [
  "matt.motley@enron.com"
 ], 
 "Content-Transfer-Encoding": "7bit", 
 "X-bcc": "", 
 "parts": [
  {
   "content": "Williams-Sonoma  \n\t\nAsparagus Plates \t  Fresh for Easter \n\nAs delicious to behold as it is to savor, asparagus heralds spring in the most delightful way. Celebrate Easter with our charming asparagus plates, along with a simple recipe for preparing the tender, elegant spears (roast them!). Happy Easter!\n\nWilliams-Sonoma\n\n\n   Asparagus Plates \n Recipe: Roasted Asparagus with Lemon \n Easter Menus \t\nRecipe: Roasted Asparagus with Lemon \n\n \t\n\n\n\nIf you prefer not to receive promotional email messages from us, please reply to this message with \"remove\" on the subject line. Your name will be removed from our email list.\n\nPrivacy Guarantee: Williams-Sonoma supports and adheres to the guideline

Messages from a query by date range:

In [9]:
start_date = dt(2001, 4, 1)
end_date = dt(2001, 4, 2)
msgs = [msg for msg in mbox.find({"Date": {
    "$lt": end_date,
    "$gt": start_date
}
}).sort("date")]
print(json.dumps(msgs, indent=1, default=json_util.default))

[
 {
  "X-cc": "", 
  "From": "spisano@sprintmail.com", 
  "Subject": "House repair bid", 
  "To": [
   "kevin.ruscitti@enron.com"
  ], 
  "Content-Transfer-Encoding": "7bit", 
  "X-bcc": "", 
  "parts": [
   {
    "content": "\n \n - RUSCITTI BID.wps \n\n", 
    "contentType": "text/plain"
   }
  ], 
  "X-Folder": "\\Ruscitti, Kevin\\Ruscitti, Kevin\\Inbox", 
  "X-Origin": "RUSCITTI-K", 
  "X-FileName": "Ruscitti, Kevin.pst", 
  "Message-ID": "<8472651.1075845282216.JavaMail.evans@thyme>", 
  "X-From": "Steven Anthony Pisano  <spisano@sprintmail.com>", 
  "Date": {
   "$date": 986163540000
  }, 
  "X-To": "KEVIN.RUSCITTI@ENRON.COM", 
  "_id": {
   "$oid": "560ef1ebf23997f642c937cb"
  }, 
  "Content-Type": "text/plain; charset=us-ascii", 
  "Mime-Version": "1.0"
 }
]


Analyze senders and receivers

In [12]:
senders = [i for i in mbox.distinct("From")]
receivers = [i for i in mbox.distinct("To")]
cc_receivers = [i for i in mbox.distinct("Cc")]
bcc_receivers = [i for i in mbox.distinct("Bcc")]
print "Num Senders:", len(senders)
print "Num Receivers:", len(receivers)
print "Num CC Receivers:", len(cc_receivers)
print "Num BCC Receivers:", len(bcc_receivers)

senders = set(senders)
receivers = set(receivers)
cc_receivers = set(cc_receivers)
bcc_receivers = set(bcc_receivers)
# Find the number of senders who were also direct receivers
senders_intersect_receivers = senders.intersection(receivers)
# Find the senders that didn't receive any messages
senders_diff_receivers = senders.difference(receivers)
# Find the receivers that didn't send any messages
receivers_diff_senders = receivers.difference(senders)
# Find the senders who were any kind of receiver by first computing the union of all types of receivers
all_receivers = receivers.union(cc_receivers, bcc_receivers)
senders_all_receivers = senders.intersection(all_receivers)

print "Num senders in common with receivers:", len(senders_intersect_receivers)
print "Num senders who didn't receive:", len(senders_diff_receivers)
print "Num receivers who didn't send:", len(receivers_diff_senders)
print "Num senders in common with *all* receivers:", len(senders_all_receivers)

# Finding senders and receivers of messages who were Enron employees
senders = [i for i in mbox.distinct("From") if i.lower().find("@enron.com") > -1]
receivers = [i for i in mbox.distinct("To") if i.lower().find("@enron.com") > -1]
cc_receivers = [i for i in mbox.distinct("Cc") if i.lower().find("@enron.com") > -1]
bcc_receivers = [i for i in mbox.distinct("Bcc") if i.lower().find("@enron.com") > -1]

print
print "Enron employees:"
print "Num Senders:", len(senders)
print "Num Receivers:", len(receivers)
print "Num CC Receivers:", len(cc_receivers)
print "Num BCC Receivers:", len(bcc_receivers)

Num Senders: 7665
Num Receivers: 22162
Num CC Receivers: 6561
Num BCC Receivers: 6561
Num senders in common with receivers: 3220
Num senders who didn't receive: 4445
Num receivers who didn't send: 18942
Num senders in common with *all* receivers: 3440

Enron employees:
Num Senders: 3137
Num Receivers: 16653
Num CC Receivers: 4890
Num BCC Receivers: 4890


Counting sent/received messages for particular email addresses

In [13]:
aliases = ["kenneth.lay@enron.com", "ken_lay@enron.com", "ken.lay@enron.com",
           "kenneth_lay@enron.net", "klay@enron.com"]  # More possibilities?
to_msgs = [msg for msg in mbox.find({"To": {"$in": aliases}})]
from_msgs = [msg for msg in mbox.find({"From": {"$in": aliases}})]
print "Number of message sent to:", len(to_msgs)
print "Number of messages sent from:", len(from_msgs)

Number of message sent to: 1326
Number of messages sent from: 7


Get recipients of email sent by Kenneth Lay

In [16]:
FROM = "kenneth.lay@enron.com"
# Get the recipient lists for each message
recipients_per_msg = list(db.mbox.aggregate([
    {"$match": {"From": re.compile(r".*{0}.*".format(FROM), re.IGNORECASE)}},
    {"$project": {"From": 1, "To": 1}},
    {"$group": {"_id": "$From", "recipients": {"$addToSet": "$To"}}}
]))
recipients_per_message = recipients_per_msg[0]['recipients']
all_recipients = [recipient for message in recipients_per_message for recipient in message]
print "Num total recipients on all messages:", len(all_recipients)
print 'First 25 recipients'
all_recipients[:25]

Num total recipients on all messages: 1043
First 25 recipients


[u'mark.koenig@enron.com',
 u'j..kean@enron.com',
 u'pr<.palmer@enron.com>',
 u'james.derrick@enron.com',
 u'elizabeth.tilney@enron.com',
 u'greg.whalley@enron.com',
 u'jeffrey.mcmahon@enron.com',
 u'raymond.bowen@enron.com',
 u'tim.despain@enron.com',
 u'e..haedicke@enron.com',
 u'erica.adams@enron.com',
 u'john.addison@enron.com',
 u'matthew.almy@enron.com',
 u'hector.alviar@enron.com',
 u'chuck.ames@enron.com',
 u'matt.anderson@enron.com',
 u'james.bakondy@enron.com',
 u'hicham.benjelloun@enron.com',
 u'shelia.benke@enron.com',
 u'christina.benkert@enron.com',
 u'peter.bennett@enron.com',
 u'aaron.berutti@enron.com',
 u'laura.bosek@enron.com',
 u'edward.brady@enron.com',
 u'erika.breen@enron.com']

Calculate the number of recipients per sent message:

In [18]:
recipients_per_message_totals = sorted([len(recipients)
                                        for recipients in recipients_per_message])
recipients_per_message_totals

[1, 1, 2, 8, 85, 946]

Get number of unique recipients:

In [19]:
unique_rec = list(db.mbox.aggregate([
    {"$match": {"From": re.compile(r".*{0}.*".format(FROM), re.IGNORECASE)}},
    {"$project": {"From": 1, "To": 1}},
    {"$unwind": "$To"},
    {"$group": {"_id": "From", "recipients": {"$addToSet": "$To"}}}
]))
unique_recipients = unique_rec[0]['recipients']
print "Num unique recipients", len(unique_recipients)

Num unique recipients 916
